In [ ]:
import os
import requests
from IPython.display import display, clear_output

anthropic_key = os.environ.get("ANTHROPIC_API_KEY")
weatherkey = os.environ.get("OPENWEATHERAPIKEY")
#don't forget to put in config file later
weatherurl = "https://api.openweathermap.org/data/2.5/forecast"

from langchain_anthropic import ChatAnthropic

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

print(weatherkey)



In [ ]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    messages: Annotated[list, add_messages]
    weather: dict


In [ ]:
# Initialize your Anthropic model (Claude)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")


In [ ]:
#############################################
# Node 1: Get weather data from OpenWeatherMap
#############################################

def get_weather(state: State):
    """
    Retrieves current weather data from the OpenWeatherMap free API.
    This node uses the 'q' parameter to pass the location (e.g., "Campinas,BR")
    and returns the weather data in the state.
    Adjust to your freaking lat long
    """
    url = weatherurl
    params = {
      "lat": -10.9,
      "lon": -50.2,
      "units": "metric",
      "appid": weatherkey  # Replace with your actual API key.
}

    
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            forecast_data = response.json()
            #print("Forecast data:", forecast_data)
            #state["weather"] = forecast_data
            state["messages"].append(f"Forecast data: {forecast_data}")
        else:
            error_msg = f"Error: {response.status_code} {response.text}"
            print(error_msg)
            state["weather"] = {"error": response.text}
            state["messages"].append(error_msg)
    except Exception as e:
        error_msg = f"Exception occurred: {e}"
        print(error_msg)
        state["weather"] = {"error": str(e)}
        state["messages"].append(error_msg)
        
    
    return state


In [ ]:
###################################################
# Node 2: Analyze weather data using Claude 
###################################################
def analyze_weather(state: State) -> State:
    """
    Sends the weather data to Claude for analysis.
    Claude returns a natural language summary tuned for an agricultural perspective.
    """
    #weather_data = state.get("weather", {})
    last_message = state["messages"][-1]

    forecast_str = last_message.content if hasattr(last_message, "content") else str(last_message)


    prompt = (
        "Please read the following weather forecast data in JSON format and shortly in one paragraph tell me the temperature for the day"
        #"Please read the following weather forecast data in JSON format and answer with one word if it will be cloudy or sunny"
        "that includes key insights and recommendations for soybean agriculture:\n\n"
        f"{forecast_str}"
    )
    
    # Use Anthropic's model to generate a summary.
    analysis = llm.invoke(prompt)

    # Extract only the content portion.
    analysis_content = analysis.content if hasattr(analysis, "content") else str(analysis)


    print("Claude Analysis:", analysis_content)
    state["messages"].append("Claude provided analysis: " + analysis_content)
    return state

In [ ]:
#############################################
# LangGraph pipeline
#############################################
graph_builder = StateGraph(State)
graph_builder.add_node("get_weather", get_weather)
graph_builder.add_node("analyze_weather", analyze_weather)

# Define the flow: start with weather retrieval then analyze.
graph_builder.set_entry_point("get_weather")
graph_builder.add_edge("get_weather", "analyze_weather")
graph_builder.add_edge("analyze_weather", END)

# Compile the graph.
graph = graph_builder.compile()

In [ ]:
#### TO SEE THE GRAPH #####

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
#############################################
# Run the graph for testing (no front-end integration)
#############################################
if __name__ == "__main__":
    # Initialize the state with an empty messages list.
    initial_state = {
        "messages": [],
        "weather": {}
    }
    
final_state = None
for state in graph.stream(initial_state):
    final_state = state  # Keep overwriting until the final state is reached

# Now print only the summary from Claude (assuming it's stored under 'analysis')
print("\n--- Final Summary from Claude ---")
print(final_state.get("analysis_content", "No summary available"))


In [ ]:
final_state = None
for state in graph.stream(initial_state):
    final_state = state  # Keep overwriting until the final state is reached

# Now print only the summary from Claude (assuming it's stored under 'analysis')
print("\n--- Final Summary from Claude ---")
print(final_state.get("analysis", "No summary available"))

In [ ]:
# Testing
if __name__ == '__main__':
    # Create an initial state dictionary.
    state = {
        "local": "Jandaia,BR",  
        "messages": []
    }
    
    # Call the get_weather function.
    updated_state = get_weather(state)
    
    # Print the final state to see the result.
    print("\nFinal state:")
    print(updated_state)

TO DO - ADD SATTELITE IMAGES

In [ ]:
#TO DO

# Node 3: Retrieve a satellite image via an external API.
# teh satellite image will come from: https://agromonitoring.com/
# get NDVI, EVI, EVI2, NRI  and weather


def get_satellite_image(state: State):
    """
    Retrieves a satellite image for the provided location.
    Replace the URL and parameters with those of your chosen satellite image API.
    """
    satellite_api_url = "https://api.satellite-imagery.com/getImage"  # Placeholder endpoint.
    params = {
        "location": state.get("local", "Campinas, BR"),
        "format": "json"
    }
    
    try:
        response = requests.get(satellite_api_url, params=params)
        satellite_data = response.json()
        print("Satellite API result:", satellite_data)
    except Exception as e:
        satellite_data = {"error": f"Satellite image fetch error: {e}"}
        print("Error fetching satellite image:", satellite_data)
    
    state["messages"].append("Satellite image retrieved via API.")
    return {"messages": state["messages"], "satellite": satellite_data}

In [ ]:
# Adding nodes to the graph.
graph_builder.add_node("weather", get_weather)
graph_builder.add_node("satellite", get_satellite_image)

In [ ]:
#add a entry point
graph_builder.add_edge(START, "chatbot")

#add an exit point

graph_builder.add_edge("chatbot", END)

In [ ]:
#compile the graph

graph = graph_builder.compile()

In [ ]:
#to see the graph

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass